## 准备工作

### bing壁纸API及网页元素分析

词组与时间，可行（打算待做功能，返回结果下载）

```
http://www.isummer.cn/#/search?searchVal=森林
```

```
http://www.isummer.cn/#/search?searchDate=20230429
```

格式固定，可作为重命名的选项。

```html
<div data-v-61a4e448="" class="x_item_description">
    <h3 data-v-61a4e448="">约书亚树国家公园上空的银河，美国加利福尼亚州(© Schroptschop/Getty Images)</h3>
    </div>
```

图片链接元素定位，将 800x480 换成 1920x1080，可行。

```
<img data-v-61a4e448="" src="https://cn.bing.com/th?id=OHR.RebelBase_ZH-CN0484516261_800x480.jpg" alt="" class="x_item_img">
```

不过，最后选型是json

![](https://s2.xptou.com/2023/05/05/6454e0e1c1302.png)



## 学习过程

直接使用原始的图片地址作为图片的下载地址和本地保存名称，可能会带有一些额外的参数或者随机字符串，导致无法正确下载或者保存图片。同时，如果直接使用原始的图片地址作为保存名称，可能会导致文件名过长或者包含一些非法字符等问题。因此，对于这种情况，最好先分割出实际的图片下载地址，再根据需要合并得到本地保存名称。

request.json()，它返回HTTP请求正文解析后的JSON对象，使用request.json()方法可以将其转化为Python字典对象。

```python
# 循环获取返回的图片地址
for i in request.json()["data"]["list"]:
    # "wpUrl" 字符串中提取出第一个以 & 符号为分隔符的第一个字符串
    picture_list = i["wpUrl"].split("&")[0]
    # print(picture_list) 返回结果：/th?id=OHR.SouthPadre_ZH-CN8788572569_1920x1080.jpg
    # 采取策略：先分割后拼接
    picture_name = picture_list.split(".") # 返回结果：['/th?id=OHR', 'SouthPadre_ZH-CN8788572569_1920x1080', 'jpg']
    print(picture_name)
```

通过 [3/library/re.split](https://docs.python.org/zh-cn/3/library/re.html#re.split)，将列表字串进行进一步分割，将“.”和“_”分割。

```python
picture_name=re.split("[._]", picture_list) # 返回结果：['/th?id=OHR', 'SouthPadre', 'ZH-CN8788572569', '1920x1080', 'jpg']
```

In [6]:
import requests
import re
import os

# 模拟浏览器请求
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}

# getWpList json 部分参数
# data = {
#     "pageNum": 1,
#     "pageSize": 9
# }

# getWpList json 部分参数
for i in range(3):
    data = {
        "pageNum": i,
        "pageSize": 9
}

    # 请求图片网站API，调用json参数
    request = requests.post('http://www.isummer.cn/x_site/wp/getWpList', json=data, headers=headers)


    # 拼接用户主目录下的 Pictures 文件夹路径
    default_pictures_dir = os.path.join(os.path.expanduser("~"), "Pictures")
    # 拼接成指定保存的图片目录
    picture_path = os.path.join(default_pictures_dir, "bing")
    # 如果目录不存在则创建
    if not os.path.exists(picture_path):
        os.makedirs(picture_path)

    # 测试现象
    # /th?id=OHR.Popocatepetl_ZH-CN5483138337_1920x1080.jpg&rf=LaDigue_1920x1080.jpg&pid=hp

    # 循环获取返回的图片地址
    for i in request.json()["data"]["list"]:
        # "wpUrl" 字符串中提取出第一个以 & 符号为分隔符的第一个字符串
        picture_list = i["wpUrl"].split("&")[0]
        # print(picture_list) 返回结果：/th?id=OHR.SouthPadre_ZH-CN8788572569_1920x1080.jpg
        # 采取策略：先分割后拼接
        # picture_name = picture_list.split(".") # 返回结果：['/th?id=OHR', 'SouthPadre_ZH-CN8788572569_1920x1080', 'jpg']
        picture_name=re.split("[._]", picture_list)

        # 拼接图片保存路径
        save_path = os.path.join(picture_path, f"{picture_name[1]}.{picture_name[4]}")
        
        # 请求图片的下载地址
        request = requests.post(f'https://cn.bing.com{picture_list}')
        # 保存图片到本地
        with open(save_path, "wb") as f:
            f.write(request.content)

        print("正在下载:" + save_path)


ModuleNotFoundError: No module named 'requests'